In [2]:
!pip install mysql-connector-python

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 31.6/31.6 MB 49.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 409.8/409.8 kB 41.9 MB/s eta 0:00:00
  Attempting uninstall: protobuf
    Found existing installation: protobuf 3.20.3
    Uninstalling protobuf-3.20.3:
      Successfully uninstalled protobuf-3.20.3
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
tensorflow-metadata 1.14.0 requires protobuf<4.21,>=3.20.3, but you have protobuf 4.21.12 which is incompatible.


In [4]:
!pip install Flask

In [3]:
import mysql.connector
import pandas as pd
from sklearn.ensemble import IsolationForest
from sklearn.preprocessing import StandardScaler
import time
import warnings
warnings.filterwarnings("ignore")

In [7]:
# Function to update the model with new data
def update_model(model, new_data, scaler):
    new_data['post_date'] = pd.to_datetime(new_data['post_date'])
    new_data.set_index('post_date', inplace=True)

    # Check if the scaler is fitted
    if not hasattr(scaler, 'mean_'):
        training_data = pd.DataFrame({'quantity': [0]})
        scaler.fit(training_data[['quantity']])

    new_data_normalized = new_data.copy()
    new_data_normalized['quantity'] = scaler.transform(new_data['quantity'].values.reshape(-1, 1))

    model = IsolationForest(contamination=0.05)
    model.fit(new_data_normalized[['quantity']])

    return model

# Function to check anomalies in a specific date
def check_anomalies_specific_date(model, data, scaler, specific_date):
    specific_data = data[data.index.date == specific_date]
    specific_data['quantity'] = scaler.transform(specific_data['quantity'].values.reshape(-1, 1))
    specific_data['anomaly'] = model.predict(specific_data[['quantity']])
    anomalies = specific_data[specific_data['anomaly'] == -1]
    return anomalies

# Function to calculate moving average and standard deviation
def calculate_moving_stats(data, window_size):
    moving_avg = data['quantity'].rolling(window=window_size).mean()
    moving_std = data['quantity'].rolling(window=window_size).std()
    return moving_avg, moving_std

# Connect to the MySQL database
conn = mysql.connector.connect(
    host="171.244.188.27",
    database="data_sky007v2",
    user="actsonevn",
    password="actsone123!"
)

# Initialize the scaler and the model
scaler = StandardScaler()
model = IsolationForest(contamination=0.05)

# Main loop to continuously fetch and process new data
while True:
    sql_query = "SELECT \
        MAX(CASE WHEN meta_key = '_product_id' THEN meta_value END) AS item_id, \
        MAX(CASE WHEN meta_key = '_qty' THEN meta_value END) AS quantity, \
        p.post_date \
    FROM aowp_posts AS p \
    JOIN aowp_woocommerce_order_items AS oi ON p.ID = oi.order_id \
    JOIN `aowp_woocommerce_order_itemmeta` AS oim ON oi.order_item_id = oim.order_item_id \
    WHERE p.post_date BETWEEN '2023-11-01 00:00:00' AND NOW() \
    GROUP BY p.id"

    new_data = pd.read_sql_query(sql_query, conn)
    model = update_model(model, new_data, scaler)

    # Check anomalies for a specific date
    specific_date = pd.to_datetime('2023-11-15')  # Replace with the desired date
    anomalies_specific_date = check_anomalies_specific_date(model, new_data, scaler, specific_date)

    # Calculate moving averages and standard deviations
    window_size = 5  # Adjust as needed
    moving_avg, moving_std = calculate_moving_stats(new_data, window_size)

    # Optionally, perform real-time anomaly detection and take actions based on the results
    anomaly_predictions = model.predict(new_data[['quantity']])
    new_data['anomaly'] = anomaly_predictions
    anomalies = new_data.loc[anomaly_predictions == -1, ['quantity']]

    # Print anomalies for a specific date
    print(f"Anomalies for {specific_date}:")
    print(anomalies_specific_date)

    # Print moving averages and standard deviations
    print(f"Moving Averages (Window Size: {window_size}):")
    print(moving_avg)
    print(f"Moving Standard Deviations (Window Size: {window_size}):")
    print(moving_std)

    # Sleep for a specific interval before fetching new data again
    time.sleep(60)  # Sleep for 60 seconds (adjust as needed)


Anomalies for 2023-11-15 00:00:00:
                     item_id  quantity  anomaly
post_date                                      
2023-11-15 00:03:38    34416       2.0       -1
2023-11-15 00:08:47   970747       2.0       -1
2023-11-15 00:25:21   970747       2.0       -1
2023-11-15 01:15:16   668026       3.0       -1
2023-11-15 01:17:35   668025       3.0       -1
2023-11-15 07:35:44   232308       2.0       -1
2023-11-15 07:55:48   232309       2.0       -1
2023-11-15 08:24:54   232309       2.0       -1
2023-11-15 09:08:15    34416       2.0       -1
2023-11-15 09:11:06   872932       6.0       -1
2023-11-15 09:13:36  1032482      10.0       -1
2023-11-15 09:16:40   872932       6.0       -1
2023-11-15 09:24:43    34416       2.0       -1
2023-11-15 10:11:02    34416       3.0       -1
2023-11-15 10:32:57  1266693       2.0       -1
2023-11-15 10:32:57    34418       3.0       -1
2023-11-15 10:50:18  1157238       5.0       -1
2023-11-15 10:54:06   232308       2.0       -1
2023-

KeyboardInterrupt: ignored

In [5]:
from flask import Flask, jsonify, request
import mysql.connector
import pandas as pd
from sklearn.ensemble import IsolationForest
from sklearn.preprocessing import StandardScaler

app = Flask(__name__)

# Initialize the scaler and the model
scaler = StandardScaler()
model = IsolationForest(contamination=0.05)

# Function to update the model with new data
def update_model(model, new_data, scaler):
    new_data['post_date'] = pd.to_datetime(new_data['post_date'])
    new_data.set_index('post_date', inplace=True)

    # Check if the scaler is fitted
    if not hasattr(scaler, 'mean_'):
        training_data = pd.DataFrame({'quantity': [0]})
        scaler.fit(training_data[['quantity']])

    new_data_normalized = new_data.copy()
    new_data_normalized['quantity'] = scaler.transform(new_data['quantity'].values.reshape(-1, 1))

    model = IsolationForest(contamination=0.05)
    model.fit(new_data_normalized[['quantity']])

    return model

# Function to check anomalies for a specific date
def check_anomalies_specific_date(model, data, scaler, specific_date):
    specific_data = data[data.index.date == specific_date]
    specific_data['quantity'] = scaler.transform(specific_data['quantity'].values.reshape(-1, 1))
    specific_data['anomaly'] = model.predict(specific_data[['quantity']])
    anomalies = specific_data[specific_data['anomaly'] == -1]
    return anomalies

# API endpoint to check anomalies for a specific date
@app.route('/check_anomalies', methods=['POST'])
def check_anomalies():
    try:
        # Get specific date from the request
        specific_date_str = request.json['specific_date']
        specific_date = pd.to_datetime(specific_date_str)

        # Fetch new data from the database
        sql_query = "SELECT \
            MAX(CASE WHEN meta_key = '_product_id' THEN meta_value END) AS item_id, \
            MAX(CASE WHEN meta_key = '_qty' THEN meta_value END) AS quantity, \
            p.post_date \
        FROM aowp_posts AS p \
        JOIN aowp_woocommerce_order_items AS oi ON p.ID = oi.order_id \
        JOIN `aowp_woocommerce_order_itemmeta` AS oim ON oi.order_item_id = oim.order_item_id \
        WHERE p.post_date BETWEEN '2023-11-01 00:00:00' AND NOW() \
        GROUP BY p.id"

        new_data = pd.read_sql_query(sql_query, conn)

        # Update the model with new data
        model = update_model(model, new_data, scaler)

        # Check anomalies for the specific date
        anomalies_specific_date = check_anomalies_specific_date(model, new_data, scaler, specific_date)

        # Return anomalies as JSON response
        return jsonify(anomalies_specific_date.to_dict(orient='records'))
    except Exception as e:
        return jsonify({'error': str(e)})

if __name__ == '__main__':
    # Connect to the MySQL database
    conn = mysql.connector.connect(
        host="171.244.188.27",
        database="data_sky007v2",
        user="actsonevn",
        password="actsone123!"
    )

    # Run the Flask app
    app.run(debug=True)


 * Serving Flask app '__main__'
 * Debug mode: on


INFO:werkzeug:WARNING: This is a development server. Do not use it in a production deployment. Use a production WSGI server instead.
 * Running on http://127.0.0.1:5000
INFO:werkzeug:Press CTRL+C to quit
INFO:werkzeug: * Restarting with stat
